# What the Func?

What is the need to have Func classes and why not just use functions? What are the additional features that Func classes provide? What are the best
 practices to use Func classes? Here we provide answers to these questions.

## Environment Setup

Here we first create a dataframe of simulated returns which will be used to demonstrate the features of Func classes.

In [1]:
from zpmeta.sources.panelsource import PanelSource
from zpmeta.funcs.func import Func
from zpmeta.singletons.singletons import MultitonMeta
from pandas import DataFrame, Series, concat, MultiIndex, date_range, IndexSlice
import numpy as np
from datetime import datetime
import logging

logging.basicConfig(level=logging.INFO)

In [2]:
class RandomReturns(PanelSource, metaclass=MultitonMeta):
    '''Subclasses Su to create a dataframe of random numbers.
    Accepts a dictionary of parameters, including:
    cols: list of column names
    '''
    def __init__(self, params: dict = None):
        super(RandomReturns, self).__init__(params)
        self.appendable = dict(xs=True, ts=True)
    
    def _execute(self, entities=None, period=None):
        cols = MultiIndex.from_product([val for val in entities.values()], names=entities.keys())
        idx = date_range(period[0], period[1], freq=self.params['freq'])
        result = DataFrame(np.random.randn(len(idx), len(cols)), columns=cols, index=idx)
        
        return result
    

In [3]:
returns_source = RandomReturns(dict(freq='B'))

INFO:root:args: ({'freq': 'B'},) ; kwds: {}
INFO:root:Multiton checking registry for key: (<class '__main__.RandomReturns'>, '[{"freq": "B"}]')
INFO:root:Multiton No Instance of <class '__main__.RandomReturns'> [{"freq": "B"}]
INFO:root:Multiton Registering Instance of <class '__main__.RandomReturns'> [{"freq": "B"}]


Let us print the dataframe

In [4]:
returns_df = returns_source(entities=dict(Type=['A','B','C'], ID=[1,2]), period=(datetime(2019,1,12), datetime(2019,1,31)))
returns_df


INFO:root:RUN RandomReturns {'freq': 'B'}
INFO:root:RUN INITIAL: [{'Type': ['A', 'B', 'C'], 'ID': [1, 2]}] 2019-01-12 00:00:00 - 2019-01-31 00:00:00
INFO:root:EXEC INITIAL: [{'Type': ['A', 'B', 'C'], 'ID': [1, 2]}] 2019-01-12 00:00:00 - 2019-01-31 00:00:00
INFO:root:DONE RandomReturns {'freq': 'B'}


Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14  0.024588 -1.227844 -0.028166 -0.620720  0.733272 -0.812820
2019-01-15  0.165194  0.211131 -1.335635 -0.162569  0.453139  0.735287
2019-01-16 -0.268999 -0.046242  0.642294  1.095727 -0.585104 -1.857545
2019-01-17  0.355699  0.793104 -0.821447 -1.088042 -0.162396 -1.077988
2019-01-18 -2.424699  0.727093 -0.298094 -0.095451  1.185986  0.656399
2019-01-21 -1.708694  0.243274  0.092338  1.588460  0.140007 -1.040684
2019-01-22 -0.234414 -0.269426 -0.865951  1.575313  0.983208 -0.181927
2019-01-23 -0.457994  2.047194  0.915780 -0.522088 -0.044318 -0.711039
2019-01-24  0.601945 -0.428440 -0.562348  0.642134 -0.916988 -0.795873
2019-01-25 -0.659989  0.270196 -0.237115 -0.609441  0.878641 -1.601735
2019-01-28 -0.452823 -1.497136 -0.810277 -1.248586  1.230389 -0.813083
2019-01-29 -0.779544 -0.042948 -0.695349 -0.621502 -1.635958  0.266640
2019-01-30 -0.751318 -1.062634 -1.096238  0.743145 -0.855565 -0.254767
2019-01-31 -0.418586 -0.447138 -0.317123 -2.154474 -0.207347 -0.105016

## Usage Example

Let us create a Func class that calculates rolling volatility of returns. It can be used to calculate both linear and exponential rolling volatility.

In [5]:
class RollingVolatility_g_Returns(Func):
    '''Calculates rolling volatility of returns. It can be used to calculate both linear and exponential rolling volatility.'''
    @classmethod
    def _std_params(cls, name: str = None) -> dict:
        return dict(type='lin',lookback = 252)

    @classmethod
    def _execute(cls, operand: DataFrame, params=None) -> DataFrame:
        if params['type'] == 'exp':
            # calculate the rolling exponential volatility
            result = operand.ewm(span=params['lookback']).std()
        elif params['type'] == 'lin':
            # calculate rolling standard  on the operand dataframe
            result = operand.rolling(params['lookback']).std()
        else:
            raise ValueError(f'given type not implemented. type must be either exp or lin')
        
        return result

Now let us use this Func class to calculate rolling volatility of the returns dataframe we created earlier.

In [6]:
vol_func = RollingVolatility_g_Returns(dict(type='lin', lookback=5))

INFO:root:INIT RollingVolatility_g_Returns {'type': 'lin', 'lookback': 5}


This 'vol_func' instance is callable, and it behaves like a usual function. Let us call it on the returns dataframe.

In [7]:
vol = vol_func(returns_df)
vol

Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-15       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-16       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-17       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-18  1.138308  0.817034  0.755480  0.814484  0.705380  1.132910
2019-01-21  1.225945  0.360507  0.770806  1.070640  0.668462  1.154459
2019-01-22  1.160502  0.467002  0.636719  1.173245  0.753956  0.962688
2019-01-23  1.139481  0.862029  0.736484  1.229336  0.619901  0.725463
2019-01-24  1.210130  0.994837  0.689014  0.957301  0.847397  0.675722
2019-01-25  0.832584  0.985563  0.686131  1.075951  0.772106  0.516891
2019-01-28  0.494496  1.299863  0.729875  1.128230  0.892640  0.507628
2019-01-29  0.549781  1.290977  0.700916  0.687046  1.199727  0.664230
2019-01-30  0.579096  0.725334  0.316406  0.872034  1.244724  0.698138
2019-01-31  0.167727  0.724584  0.355806  1.058645  1.190339  0.727275

We don't need to create an explicit instance variable to call the Func class. We can call it directly as follows:

In [9]:
vol = RollingVolatility_g_Returns(dict(type='lin', lookback=5))(returns_df)
vol

INFO:root:INIT RollingVolatility_g_Returns {'type': 'lin', 'lookback': 5}


Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-15       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-16       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-17       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-18  1.138308  0.817034  0.755480  0.814484  0.705380  1.132910
2019-01-21  1.225945  0.360507  0.770806  1.070640  0.668462  1.154459
2019-01-22  1.160502  0.467002  0.636719  1.173245  0.753956  0.962688
2019-01-23  1.139481  0.862029  0.736484  1.229336  0.619901  0.725463
2019-01-24  1.210130  0.994837  0.689014  0.957301  0.847397  0.675722
2019-01-25  0.832584  0.985563  0.686131  1.075951  0.772106  0.516891
2019-01-28  0.494496  1.299863  0.729875  1.128230  0.892640  0.507628
2019-01-29  0.549781  1.290977  0.700916  0.687046  1.199727  0.664230
2019-01-30  0.579096  0.725334  0.316406  0.872034  1.244724  0.698138
2019-01-31  0.167727  0.724584  0.355806  1.058645  1.190339  0.727275

## Feature 1: Default Params

Using the _std_params method, we can define default parameters for the Func class. These default parameters are used if the user does not provide 
the params while calling the Func instance. In the above example, we have defined the default params as type='lin' and lookback=252. If the user 
does not provide these params while calling the Func instance, it returns the rolling linear volatility with a lookback of 252 days.

Default params are an in-built feature in Python, what else can we do with Funcs? We can provide named default params. Let us define a Func 
class that calculates the rolling volatility as per the RiskMetrics methodology. Let us redefine our class as follows:

In [10]:
class RollingVolatility_g_Returns(Func):
    '''Calculates rolling volatility of returns. It can be used to calculate both linear and exponential rolling volatility.'''
    @classmethod
    def _std_params(cls, name: str = None) -> dict:
        if name is None:
            return dict(type='lin',lookback = 252)
        elif name == 'RiskMetrics':
            return dict(type='exp',lookback = 33)
        elif name == 'DRQ':
            return dict(type='lin',lookback = 5)
        else:
            raise ValueError(f'given name not implemented. name must be either None or RiskMetrics')
        
        return dict(type='lin',lookback = 252)

    @classmethod
    def _execute(cls, operand: DataFrame, params=None) -> DataFrame:
        if params['type'] == 'exp':
            # calculate the rolling exponential volatility
            result = operand.ewm(span=params['lookback']).std()
        elif params['type'] == 'lin':
            # calculate rolling standard  on the operand dataframe
            result = operand.rolling(params['lookback']).std()
        else:
            raise ValueError(f'given type not implemented. type must be either exp or lin')
        
        return result

Now let us see the power of default parameters.

In [11]:
vol_func = RollingVolatility_g_Returns(params=('RiskMetrics',{}))
vol = vol_func(returns_df)
vol

INFO:root:INIT RollingVolatility_g_Returns {'type': 'exp', 'lookback': 33}


Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-15  0.099424  1.017509  0.924520  0.323962  0.198084  1.094677
2019-01-16  0.225364  0.754015  1.017148  0.896299  0.701626  1.321440
2019-01-17  0.269758  0.838057  0.873233  0.958312  0.589988  1.083192
2019-01-18  1.189799  0.795978  0.745675  0.818357  0.720537  1.141404
2019-01-21  1.173020  0.702038  0.686079  1.048975  0.639656  1.036183
2019-01-22  1.072420  0.654833  0.660811  1.102645  0.639341  0.944084
2019-01-23  0.979494  0.961649  0.782031  1.066295  0.608282  0.863774
2019-01-24  1.006399  0.934949  0.733280  0.991145  0.726837  0.800713
2019-01-25  0.936530  0.867097  0.680062  0.976069  0.719945  0.827501
2019-01-28  0.873862  1.011367  0.664453  1.036620  0.749859  0.773000
2019-01-29  0.825841  0.948904  0.636416  0.995229  0.970077  0.795219
2019-01-30  0.781537  0.962953  0.646082  0.971488  0.969035  0.757173
2019-01-31  0.739030  0.917257  0.611271  1.149036  0.918558  0.729971

As we see, just by providing the 'RiskMetrics' label, the Func class now uses the RiskMetrics parameters. Of course, we can do this using regular 
functions, but Func allows us to remember multiple parameters for a given function. This is useful when we have multiple parameters that we want. 
Let us call it again using the 'DRQ' setting.

In [12]:
drq_vol_func = RollingVolatility_g_Returns(params=('DRQ',{}))
vol = drq_vol_func(returns_df)
vol

INFO:root:INIT RollingVolatility_g_Returns {'type': 'lin', 'lookback': 5}


Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-15       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-16       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-17       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-18  1.138308  0.817034  0.755480  0.814484  0.705380  1.132910
2019-01-21  1.225945  0.360507  0.770806  1.070640  0.668462  1.154459
2019-01-22  1.160502  0.467002  0.636719  1.173245  0.753956  0.962688
2019-01-23  1.139481  0.862029  0.736484  1.229336  0.619901  0.725463
2019-01-24  1.210130  0.994837  0.689014  0.957301  0.847397  0.675722
2019-01-25  0.832584  0.985563  0.686131  1.075951  0.772106  0.516891
2019-01-28  0.494496  1.299863  0.729875  1.128230  0.892640  0.507628
2019-01-29  0.549781  1.290977  0.700916  0.687046  1.199727  0.664230
2019-01-30  0.579096  0.725334  0.316406  0.872034  1.244724  0.698138
2019-01-31  0.167727  0.724584  0.355806  1.058645  1.190339  0.727275

We can use the named params while overriding some of the elements of the parameters. For example, let us use the RiskMetrics parameters but 
override the lookback to 66 days.

In [57]:
annual_drq_vol_func = RollingVolatility_g_Returns(params=('DRQ',{'lookback': 252}))
vol = annual_drq_vol_func(returns_df)
vol

INFO:root:INIT RollingVolatility_g_Returns {'type': 'lin', 'lookback': 252}


Type         A       B       C    
ID           1   2   1   2   1   2
2019-01-14 NaN NaN NaN NaN NaN NaN
2019-01-15 NaN NaN NaN NaN NaN NaN
2019-01-16 NaN NaN NaN NaN NaN NaN
2019-01-17 NaN NaN NaN NaN NaN NaN
2019-01-18 NaN NaN NaN NaN NaN NaN
2019-01-21 NaN NaN NaN NaN NaN NaN
2019-01-22 NaN NaN NaN NaN NaN NaN
2019-01-23 NaN NaN NaN NaN NaN NaN
2019-01-24 NaN NaN NaN NaN NaN NaN
2019-01-25 NaN NaN NaN NaN NaN NaN
2019-01-28 NaN NaN NaN NaN NaN NaN
2019-01-29 NaN NaN NaN NaN NaN NaN
2019-01-30 NaN NaN NaN NaN NaN NaN
2019-01-31 NaN NaN NaN NaN NaN NaN

As we can see the default DRQ params have been updated with a lookback of 252.

We can also do this overriding of params while calling the Func as a function

In [58]:
vol = annual_drq_vol_func(returns_df, params={'lookback': 11})
vol


Type         A       B       C    
ID           1   2   1   2   1   2
2019-01-14 NaN NaN NaN NaN NaN NaN
2019-01-15 NaN NaN NaN NaN NaN NaN
2019-01-16 NaN NaN NaN NaN NaN NaN
2019-01-17 NaN NaN NaN NaN NaN NaN
2019-01-18 NaN NaN NaN NaN NaN NaN
2019-01-21 NaN NaN NaN NaN NaN NaN
2019-01-22 NaN NaN NaN NaN NaN NaN
2019-01-23 NaN NaN NaN NaN NaN NaN
2019-01-24 NaN NaN NaN NaN NaN NaN
2019-01-25 NaN NaN NaN NaN NaN NaN
2019-01-28 NaN NaN NaN NaN NaN NaN
2019-01-29 NaN NaN NaN NaN NaN NaN
2019-01-30 NaN NaN NaN NaN NaN NaN
2019-01-31 NaN NaN NaN NaN NaN NaN

The difference here is that this override is temporary and does not change the default params of the Func class. 

In [59]:
annual_drq_vol_func.params

{'type': 'lin', 'lookback': 252}

These features are extremely helpful when you have a long list of parameters in quantitative research, with some standardized parameters, but where 
you also need to change some of the parameters for specific use cases.

## Feature 2: Partial Functions

From the above examples, one can see that Func classes can be used to effortlessly create partial functions. Let us look at an example.

In [13]:
class RollingVolatility_g_Returns(Func):
    '''Calculates rolling volatility of returns. It can be used to calculate both linear and exponential rolling volatility.'''
    @classmethod
    def _std_params(cls, name: str = None) -> dict:
        return {}

    @classmethod
    def _execute(cls, operand: DataFrame, params=None) -> DataFrame:
        if params['type'] == 'exp':
            # calculate the rolling exponential volatility
            result = operand.ewm(span=params['lookback']).std()
        elif params['type'] == 'lin':
            # calculate rolling standard  on the operand dataframe
            result = operand.rolling(params['lookback']).std()
        else:
            raise ValueError(f'given type not implemented. type must be either exp or lin')
        
        return result
    
vol_func = RollingVolatility_g_Returns(params={'type': 'lin'})


INFO:root:INIT RollingVolatility_g_Returns {'type': 'lin'}


Now "vol_func" behaves like a partial function. We can feed it any lookback parameter 
that we want.

In [14]:
vol = vol_func(returns_df, {'lookback': 5})
vol

Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-15       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-16       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-17       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-18  1.138308  0.817034  0.755480  0.814484  0.705380  1.132910
2019-01-21  1.225945  0.360507  0.770806  1.070640  0.668462  1.154459
2019-01-22  1.160502  0.467002  0.636719  1.173245  0.753956  0.962688
2019-01-23  1.139481  0.862029  0.736484  1.229336  0.619901  0.725463
2019-01-24  1.210130  0.994837  0.689014  0.957301  0.847397  0.675722
2019-01-25  0.832584  0.985563  0.686131  1.075951  0.772106  0.516891
2019-01-28  0.494496  1.299863  0.729875  1.128230  0.892640  0.507628
2019-01-29  0.549781  1.290977  0.700916  0.687046  1.199727  0.664230
2019-01-30  0.579096  0.725334  0.316406  0.872034  1.244724  0.698138
2019-01-31  0.167727  0.724584  0.355806  1.058645  1.190339  0.727275